This is a code I used at the Data Science Hackathon in XCat where I got the second best prize out of 12 groups by myself and almost all paticipants were graduate students except for me.

This code vedoids of the data from XCat due to the confidential reasons. Therefore, this code would not work properly without the data from them. It is just a portfolio of my Causal Inference skills

Difference in Difference (DID)

DF_evaluation_revisedで一つでも欠損値を含むレコードを削除して分析

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
plt.style.use('ggplot')

import openpyxl


from datetime import datetime
%matplotlib inline

In [ ]:
cd ..

In [ ]:
#配布データの読み込み

#社員情報の読み取り
worker = pd.ExcelFile("ALL\【ハッカソン】社員情報.xlsx")
#sheet_namesメソッドでExcelブック内の各シートの名前をリストで取得できる
input_sheet_name = worker.sheet_names
DF_worker = worker.parse(input_sheet_name[0])




#評価データの読み取り
evaluation = pd.ExcelFile("ALL\【ハッカソン】評価データ.xlsx")
#sheet_namesメソッドでExcelブック内の各シートの名前をリストで取得できる
input_sheet_name = evaluation.sheet_names
DF_evaluation = evaluation.parse(input_sheet_name[0])



#評価データ（数値以外のデータを削除した加工済み）の読み取り
evaluation_revised = pd.ExcelFile("ALL\evaluation_data_get_rid_off_string.xlsx")
#sheet_namesメソッドでExcelブック内の各シートの名前をリストで取得できる
input_sheet_name = evaluation_revised.sheet_names
DF_evaluation_revised = evaluation_revised.parse(input_sheet_name[0])



#日時勤務実績2017~2021の読み取り

working2017 = pd.ExcelFile("ALL\【ハッカソン】日次勤務実績_2017年度 （20170401-20180331）.xlsx")
#sheet_namesメソッドでExcelブック内の各シートの名前をリストで取得できる
input_sheet_name = working2017.sheet_names
DF_working2017 = working2017.parse(input_sheet_name[0])


working2018 = pd.ExcelFile("ALL\【ハッカソン】日次勤務実績_2018年度（20180401-20190331）.xlsx")
#sheet_namesメソッドでExcelブック内の各シートの名前をリストで取得できる
input_sheet_name = working2018.sheet_names
DF_working2018 = working2018.parse(input_sheet_name[0])

working2019 = pd.ExcelFile("ALL\【ハッカソン】日次勤務実績_2019年度（20190401-20200331）.xlsx")
#sheet_namesメソッドでExcelブック内の各シートの名前をリストで取得できる
input_sheet_name = working2019.sheet_names
DF_working2019 = working2019.parse(input_sheet_name[0])

working2020 = pd.ExcelFile("ALL\【ハッカソン】日次勤務実績_2020年度 （20200401-20210331）.xlsx")
#sheet_namesメソッドでExcelブック内の各シートの名前をリストで取得できる
input_sheet_name = working2020.sheet_names
DF_working2020 = working2020.parse(input_sheet_name[0])

working2021 = pd.ExcelFile("ALL\【ハッカソン】日次勤務実績_2021年度 （20210401-20220331）.xlsx")
#sheet_namesメソッドでExcelブック内の各シートの名前をリストで取得できる
input_sheet_name = working2021.sheet_names
DF_working2021 = working2021.parse(input_sheet_name[0])

In [ ]:
DF_evaluation_revised = DF_evaluation_revised.dropna()

In [ ]:
# 2017~2021をまとめる

DF_working_all = pd.concat([DF_working2017, DF_working2018, DF_working2019, DF_working2020, DF_working2021], axis=0)

DF_working_all = DF_working_all.reset_index()
DF_working_all = DF_working_all.drop(columns='index')
DF_working_all

In [ ]:
DF_worker

In [ ]:
DF_evaluation

In [ ]:
DF_working_all

## テレワーク勤務の利用者数と利用割合を時系列で可視化する

In [ ]:
#勤務日でまとめて平均を出す
DF_working_mean = DF_working_all.groupby('勤務日').mean()

In [ ]:
DF_working_mean_telework = DF_working_mean["テレワーク勤務"]

In [ ]:
date_tele = list(DF_working_mean_telework.index)


In [ ]:
telework_num = list(DF_working_mean_telework)


In [ ]:
telework_num[0]

In [ ]:
with open("ALL/telework_time.txt", mode="w",  encoding="utf-8", ) as f:
    for i in range(len(date_tele)):
        words = str(date_tele[i]) + ", " + str(telework_num[i]) + "\n"
        f.write(words)

In [ ]:
time_mean = list(DF_working_mean_telework.index)

In [ ]:
plt.scatter(time_mean, DF_working_mean_telework*100, color="green")
plt.xlabel("時間", fontname="MS Gothic")
plt.ylabel("テレワーク平均利用者割合（％）", fontname="MS Gothic")
plt.title("テレワーク利用者割合（％）の散布図", fontname="MS Gothic")

In [ ]:
#勤務日でまとめて総和で出す
DF_working_sum = DF_working_all.groupby('勤務日').sum()

In [ ]:
DF_working_sum_telework = DF_working_sum["テレワーク勤務"]

In [ ]:
time_sum = list(DF_working_sum_telework.index)

plt.scatter(time_sum, DF_working_sum_telework)
plt.xlabel("時間", fontname="MS Gothic")
plt.ylabel("テレワーク総利用者数（人）", fontname="MS Gothic")
plt.title("テレワーク利用者数（人）の散布図", fontname="MS Gothic")

## 少なくとも一度テレワーク勤務を利用した人を介入群として差の差分析
期間：2022/03/31~2022/04/01

In [ ]:
from_2021 = datetime(2021, 3, 31)
to_2021 = datetime(2022, 4, 1)
DF_working_2021 = DF_working_all[(from_2021 <= DF_working_all["勤務日"]) & (DF_working_all["勤務日"] <= to_2021)]

In [ ]:
DF_working_2021_sum = DF_working_2021.groupby('社員コード').sum()

DF_working_2021_sum

In [ ]:
#テレワーク勤務を2021年に一度でも行った社員コードをリストにする
list_yes_telework = list(DF_working_2021_sum[DF_working_2021_sum["テレワーク勤務"] > 0].index)

list_no_telework = list(DF_working_2021_sum[DF_working_2021_sum["テレワーク勤務"] == 0].index)

In [ ]:
DF_evaluation_DID = DF_evaluation_revised.copy()

# 列名を変更
DF_evaluation_DID = DF_evaluation_DID.rename(columns={
                                                    "2013年度": 2013, 
                                                    "2014年度": 2014, 
                                                    "2015年度": 2015, 
                                                    "2016年度": 2016, 
                                                    "2017年度": 2017, 
                                                    "2018年度": 2018, 
                                                    "2019年度": 2019, 
                                                    "2020年度": 2020, 
                                                    "2021年度": 2021, 
                                                    })

# CGを初期値としてgroupをデータフレームに追加
DF_evaluation_DID["group"] = "CG"

list_worker_number = list(DF_evaluation_DID["社員番号"])

# 2021年の間に少なくとも1度テレワークを利用した労働者をTGとする
for i in range(len(list_worker_number)):
    worker_number = str(list_worker_number[i])
    if (worker_number in list_yes_telework):
        DF_evaluation_DID.loc[i, "group"] = "TG"

# TGとCGで集計する
DF_evaluation_DID_summary = DF_evaluation_DID.groupby("group").mean()


DF_evaluation_DID_summary


In [ ]:
# CGとTGをDIDで可視化

plt.plot(DF_evaluation_DID_summary.loc["CG"], label="比較群")
plt.plot(DF_evaluation_DID_summary.loc["TG"], label="介入群")
plt.xlabel("時間", fontname="MS Gothic")
plt.ylabel("評価（低い方が高評価）", fontname="MS Gothic")
plt.title("テレワーク勤務を一回以上行った人を介入群とした差の差分析", fontname="MS Gothic")
plt.axvline(x=2019.5, color="purple", label="テレワーク導入")
plt.legend(prop={'family':'Yu Gothic'})

In [ ]:
DF_evaluation_DID_summary

In [ ]:
# テレワーク導入の因果効果を求める

TG2021 = DF_evaluation_DID_summary.loc["TG", 2021]
TG2020 = DF_evaluation_DID_summary.loc["TG", 2020]
CG2021 = DF_evaluation_DID_summary.loc["CG", 2021]
CG2020 = DF_evaluation_DID_summary.loc["CG", 2020]

causal_effect = (TG2021 - TG2020) - (CG2021 - CG2020)

causal_effect

## 出勤日数の半分以上でテレワーク勤務を利用した人を介入群として差の差分析
期間：2022/03/31~2022/04/01

In [ ]:
DF_working_2021_sum.loc[:, ["出勤日数", "テレワーク勤務"]]

In [ ]:
DF_working_2021_sum["telework ratio"] = DF_working_2021_sum["テレワーク勤務"] / DF_working_2021_sum["出勤日数"]

In [ ]:
DF_tele_ratio = DF_working_2021_sum.loc[:, ["telework ratio", "出勤日数", "テレワーク勤務"]]

In [ ]:
ratios = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
telework_users = []
for i in range(len(ratios)):
    number_of_telework_users = len(DF_tele_ratio[DF_tele_ratio["telework ratio"] <= ratios[i]])
    telework_users.append(number_of_telework_users)

percentage = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
plt.plot(percentage, telework_users)
plt.xlabel("テレワーク勤務の割合（％）", fontname="MS Gothic")
plt.ylabel("その割合以下の人数", fontname="MS Gothic")
plt.title("テレワークの割合とそれ以下の人数の累積分布", fontname="MS Gothic")

In [ ]:
#テレワーク勤務が半分以上かどうかで分類し、社員コードをリストにする
list_telework_TG = list(DF_tele_ratio[DF_tele_ratio["telework ratio"] >= 0.5].index)

list_telework_CG = list(DF_tele_ratio[DF_tele_ratio["telework ratio"] < 0.5].index)

In [ ]:
list_telework_TG

In [ ]:
DF_evaluation_DID2 = DF_evaluation_revised.copy()

# 列名を変更
DF_evaluation_DID2 = DF_evaluation_DID2.rename(columns={
                                                    "2013年度": 2013, 
                                                    "2014年度": 2014, 
                                                    "2015年度": 2015, 
                                                    "2016年度": 2016, 
                                                    "2017年度": 2017, 
                                                    "2018年度": 2018, 
                                                    "2019年度": 2019, 
                                                    "2020年度": 2020, 
                                                    "2021年度": 2021, 
                                                    })

# CGを初期値としてgroupをデータフレームに追加
DF_evaluation_DID2["group"] = "CG"

list_worker_number = list(DF_evaluation_DID2["社員番号"])

# 半分以上テレワークを利用した労働者をTGとする
for i in range(len(list_worker_number)):
    worker_number = str(list_worker_number[i])
    if (worker_number in list_telework_TG):
        DF_evaluation_DID2.loc[i, "group"] = "TG"

# TGとCGで集計する
DF_evaluation_DID2_summary = DF_evaluation_DID2.groupby("group").mean()


DF_evaluation_DID2_summary


In [ ]:
list(DF_evaluation_DID2_summary.loc["CG"])

In [ ]:
# CGとTGをDIDで可視化

from tkinter.ttk import Style


plt.plot(DF_evaluation_DID2_summary.loc["CG"], label="比較群（テレワーク非利用）", color="black", linestyle="dashed")
plt.plot(DF_evaluation_DID2_summary.loc["TG"], label="介入群（テレワーク利用）", color="black")
plt.xlabel("時間", fontname="MS Gothic")
plt.ylabel("評価（低い方が高評価）", fontname="MS Gothic")
plt.title("テレワークが評価（生産性）に与える影響の差の差分析(DID)", fontname="MS Gothic")
plt.axvline(x=2019.5, color="green", label="テレワーク導入")
plt.legend(prop={'family':'Yu Gothic'})

In [ ]:
# テレワーク導入の因果効果を求める

TG_after_2 = (DF_evaluation_DID2_summary.loc["TG", 2020] + DF_evaluation_DID2_summary.loc["TG", 2021]) / 2
TG_before_2 = DF_evaluation_DID2_summary.loc["TG", 2019]
CG_after_2 = (DF_evaluation_DID2_summary.loc["CG", 2020] + DF_evaluation_DID2_summary.loc["CG", 2021]) / 2
CG_before_2 = DF_evaluation_DID2_summary.loc["CG", 2019]

causal_effect2 = (TG_after_2 - TG_before_2) - (CG_after_2 - CG_before_2)

causal_effect2

In [ ]:
print(TG_after_2, TG_before_2, CG_after_2, CG_before_2)

## 合成コントロール法をRで使用する際のデータフレームの作成

In [ ]:
DF_evaluation_DID2

In [ ]:
list(DF_evaluation_DID2[DF_evaluation_DID2["社員番号"] == "P80509313"][2015])[0]

In [ ]:
evaluation_2013 = list(DF_evaluation_DID2[2013])
evaluation_2014 = list(DF_evaluation_DID2[2014])
evaluation_2015 = list(DF_evaluation_DID2[2015])
evaluation_2016 = list(DF_evaluation_DID2[2016])
evaluation_2017 = list(DF_evaluation_DID2[2017])
evaluation_2018 = list(DF_evaluation_DID2[2018])
evaluation_2019 = list(DF_evaluation_DID2[2019])
evaluation_2020 = list(DF_evaluation_DID2[2020])
evaluation_2021 = list(DF_evaluation_DID2[2021])

workers_number = list(DF_evaluation_DID2["社員番号"])
group = list(DF_evaluation_DID2["group"])

In [ ]:
cols = ['社員番号', '年度', 'group', '評価']

year = list(range(2013, 2022))

contents = []

for i in range(len(workers_number)):
    for j in range(len(year)):
        CG_or_TG = list(DF_evaluation_DID2[DF_evaluation_DID2["社員番号"] == workers_number[i]]["group"])[0]
        evaluation = list(DF_evaluation_DID2[DF_evaluation_DID2["社員番号"] == workers_number[i]][year[j]])[0]
        contents.append([workers_number[i], year[j], CG_or_TG, evaluation])
        #data_frame = pd.DataFrame({'社員番号': workers_number[i], '年': year[j], 'group': CG_or_TG, '評価': evaluation})
        #data_frame = pd.DataFrame([workers_number[i], year[j], CG_or_TG, evaluation], columns=cols)
        

In [ ]:
DF_for_SCM = pd.DataFrame(contents, columns=cols)

In [ ]:
DF_for_SCM["group_number"] = 1

for i in range(len(DF_for_SCM)):
    if(DF_for_SCM.loc[i, "group"] == "TG"):
        DF_for_SCM.loc[i, "group_number"] = 2

In [ ]:
DF_for_SCM

In [ ]:
# groupbyで集計する際に年度列を追加

from_2017 = datetime(2017, 4, 1)
to_2018 = datetime(2018, 3, 31)

from_2018 = datetime(2018, 4, 1)
to_2019 = datetime(2019, 3, 31)

from_2019 = datetime(2019, 4, 1)
to_2020 = datetime(2020, 3, 31)

from_2020 = datetime(2020, 4, 1)
to_2021 = datetime(2021, 3, 31)

from_2021 = datetime(2021, 4, 1)
to_2022 = datetime(2022, 3, 31)

for i in range(len(DF_working_all)):
    if(from_2017 <= DF_working_all.loc[i, "勤務日"] <= to_2018):
        DF_working_all.loc[i, "勤務年度"] = int(2017)
    if(from_2018 <= DF_working_all.loc[i, "勤務日"] <= to_2019):
        DF_working_all.loc[i, "勤務年度"] = int(2018)
    if(from_2019 <= DF_working_all.loc[i, "勤務日"] <= to_2020):
        DF_working_all.loc[i, "勤務年度"] = int(2019)
    if(from_2020 <= DF_working_all.loc[i, "勤務日"] <= to_2021):
        DF_working_all.loc[i, "勤務年度"] = int(2020)
    if(from_2021 <= DF_working_all.loc[i, "勤務日"] <= to_2022):
        DF_working_all.loc[i, "勤務年度"] = int(2021)

DF_working_all

In [ ]:
# 社員コードと勤務年度で集計
DF_working_summarized = DF_working_all.groupby(["社員コード", "勤務年度"]).mean()

In [ ]:
# 後でデータフレームの結合を行う時のために列名の変更を行う
DF_working_summarized.reset_index(inplace= True)

DF_working_summarized = DF_working_summarized.rename(columns={'社員コード': '社員番号', 
                                                              '勤務年度': '年度'})

DF_working_summarized

In [ ]:
DF_working_summarized.columns

In [ ]:
# 平均を求めるべきでない列を削除
DF_working_summarized = DF_working_summarized.drop(columns=["勤務区分No", "休暇区分No"])

In [ ]:
DF_working_summarized.columns

In [ ]:
DF_for_SCM

In [ ]:
DF_for_SCM_merged = pd.merge(DF_for_SCM, DF_working_summarized, on=["社員番号", "年度"], how="left")

In [ ]:
DF_for_SCM_merged

In [ ]:
# Rで使用するSCMの特性上、社員番号に対して番号を振り当てる準備を行う

# 社員番号を一意な形でリストにする
worker_numbers = list(DF_for_SCM_merged["社員番号"])
worker_numbers_unique = []

for i in range(len(worker_numbers)):
    if(worker_numbers[i] not in worker_numbers_unique):
        worker_numbers_unique.append(worker_numbers[i])

# 社員番号に一意なIDを振り当てた二次元上のリストを作成
worker_numbers_unique_with_ID = []

for i in range(len(worker_numbers_unique)):
    list_workers_and_workers_ID = [worker_numbers_unique[i], i+1]
    worker_numbers_unique_with_ID.append(list_workers_and_workers_ID)

worker_numbers_unique_with_ID


# 作成した社員IDをDF_for_SCM_mergedの列に追加する
for i in range(len(worker_numbers_unique_with_ID)):
    for j in range(len(DF_for_SCM_merged)):
        if(DF_for_SCM_merged.loc[j, "社員番号"] == worker_numbers_unique_with_ID[i][0]):
            DF_for_SCM_merged.loc[j, "社員ID"] = worker_numbers_unique_with_ID[i][1]

In [ ]:
DF_for_SCM_merged

In [ ]:
workers_ID_TG = list(set(DF_for_SCM_merged[DF_for_SCM_merged["group"] == "TG"]["社員ID"]))

workers_ID_TG

In [ ]:
workers_ID_CG = list(set(DF_for_SCM_merged[DF_for_SCM_merged["group"] == "CG"]["社員ID"]))

workers_ID_CG

In [ ]:
for i in range(len(workers_ID_TG)):
    for j in range(len(DF_for_SCM_merged)):
        if(DF_for_SCM_merged.loc[j, "社員ID"] == workers_ID_TG[i]):
            DF_for_SCM_merged.loc[j, "社員ID"] = 1000

In [ ]:
DF_for_SCM_merged

In [ ]:
DF_for_SCM_merged_revied = DF_for_SCM_merged.copy()

for i in range(len(DF_for_SCM_merged_revied)):
    if(DF_for_SCM_merged_revied.loc[i, "group"] == "TG"):
        DF_for_SCM_merged_revied.loc[i, "社員番号"] = "TG"

DF_for_SCM_merged_revied

In [ ]:
DF_for_SCM_merged_revied.columns

In [ ]:
DF_for_SCM_merged_revied = DF_for_SCM_merged_revied.drop(columns=[
                                                                    
                                                                    
                                                                    'group', 
                                                                    'group_number', 
                                                                    
                                                                    #'出勤日数', 
                                                                    #'実働時間', 
                                                                    #'休憩時間'
                                                                    ])

DF_for_SCM_merged_revied

In [ ]:
DF_for_SCM_merged_revied2 = DF_for_SCM_merged_revied.groupby(["社員番号", "年度"]).mean()

In [ ]:
DF_for_SCM_merged_revied2.reset_index(inplace= True)

DF_for_SCM_merged_revied2

In [ ]:
import csv
import pprint
 
DF_for_SCM_merged_revied2.to_csv('ALL/evaluation_for_SCM.csv', index=False)